<a href="https://colab.research.google.com/github/kofie1990/potato-diseases/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 2152 files belonging to 3 classes.


In [4]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [5]:
train_size = 0.8
len(dataset) * train_size

54.400000000000006

In [6]:
train_ds = dataset.take(54)
len(train_ds)

54

In [7]:
test_ds = dataset.skip(54)
len(dataset) * 0.1

6.800000000000001

In [8]:
val_ds = test_ds.take(6)
len(val_ds)

6

In [9]:
test_ds = test_ds.skip(6)
len(test_ds)

8

In [10]:
def get_dataset_partitions_tf(ds, train_split = 0.8, val_split=0.1, test_split=0.1,shuffle=True, shuffle_size=10000):
  ds_size = len(ds)

  if shuffle:
    ds = ds.shuffle(shuffle_size, seed=12)

  train_size = int(train_split * ds_size)
  val_size = int(val_split * ds_size)

  train_ds = ds.take(train_size)

  val_ds  = ds.skip(train_size).take(val_size)
  test_ds  = ds.skip(train_size).skip(val_size)

  return train_ds, val_ds, test_ds

In [11]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [12]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [14]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [15]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, (3,3), activation='relu', input_shape= input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (32, 127, 127, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (32, 62, 62, 64)          0         
 g2D)                                                 

In [17]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [18]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/50
54/54 [==============================] - 361s 175ms/step - loss: 0.9163 - accuracy: 0.5069 - val_loss: 0.9705 - val_accuracy: 0.4271
Epoch 2/50
54/54 [==============================] - 3s 62ms/step - loss: 0.6996 - accuracy: 0.6713 - val_loss: 0.6124 - val_accuracy: 0.7396
Epoch 3/50
54/54 [==============================] - 3s 59ms/step - loss: 0.3824 - accuracy: 0.8449 - val_loss: 0.2067 - val_accuracy: 0.9219
Epoch 4/50
54/54 [==============================] - 3s 59ms/step - loss: 0.2076 - accuracy: 0.9277 - val_loss: 0.8857 - val_accuracy: 0.7240
Epoch 5/50
54/54 [==============================] - 3s 62ms/step - loss: 0.2165 - accuracy: 0.9155 - val_loss: 0.2533 - val_accuracy: 0.9062
Epoch 6/50
54/54 [==============================] - 3s 60ms/step - loss: 0.1602 - accuracy: 0.9450 - val_loss: 0.4751 - val_accuracy: 0.8125
Epoch 7/50
54/54 [==============================] - 3s 59ms/step - loss: 0.1337 - accuracy: 0.9514 - val_loss: 0.2310 - val_accuracy: 0.9062
Epoch 8/50

In [19]:
model.evaluate(test_ds)

8/8 [==============================] - 6s 27ms/step - loss: 0.0224 - accuracy: 0.9922


[0.022409584373235703, 0.9921875]